In [1]:
# Filenames
file_small_1 = 'data/data-en.txt'
file_small_2 = 'data/data-pl.txt'
file_large_1 = 'data/data-en-large.txt'
file_large_2 = 'data/data-pl-large.txt'
file_emea_1 = 'data/EMEA.en-pl.en'
file_emea_2 = 'data/EMEA.en-pl.pl'

In [14]:
# Create a smaller file with sentences partially including source text.
import os, re
from nltk import bigrams

temp_file = 'data/temp.txt'
example_text = "Benefits payable from the social insurance system are financed by the Social Insurance Fund"

regex = re.compile('[^a-zA-Z\s]')
example_text_words = regex.sub('', example_text).split()
if len(example_text_words) > 1:
    words = list(bigrams(example_text_words))
    patterns = ' '.join(f"-e '{word[0]} {word[1]}'" for word in words)
else: 
    patterns = f"-e '{example_text}'"
print(patterns)
# os.system("echo %s | xargs -n1 -I{} grep -n -F {} input_file.txt > output_file.txt")
os.system("grep -n -w -i -F %s %s | cat >> %s" %(patterns, file_large_1, temp_file))


-e 'Benefits payable' -e 'payable from' -e 'from the' -e 'the social' -e 'social insurance' -e 'insurance system' -e 'system are' -e 'are financed' -e 'financed by' -e 'by the' -e 'the Social' -e 'Social Insurance' -e 'Insurance Fund'


0

In [3]:
print('Number of lines in temp_file:')
os.system("wc -l %s" % temp_file)
print('\n')
print('First few lines from temp_file:')
os.system("head %s" % temp_file)

Number of lines in temp_file:
133660 data/temp.txt


First few lines from temp_file:
400:You must be hungry, dear.
1064:You must be crazy!
1134:-He must be sensitive.
1772:It must be 'that day' for Eunchae.
1841:Instead of wearing yourself out about it, It's better to not start something that isn't meant to happen.
2007:You must be the new guy that moved in downstairs.
2532:anyway that girl must be crazy!
2620:Come on...teacher must be waiting.
3062:You must be stegnesh
3074:I must be immune.


0

In [5]:
# Utilize tfidf_matcher library to perform fuzzy matching using cosine similarity
import tfidf_matcher as tm

with open(temp_file, 'r', encoding='utf-8') as f1:
    lines = f1.readlines()
    unique_lines = set(lines)
    result = tm.matcher(original=[example_text], lookup=unique_lines, k_matches=5, ngram_length=2)
    print(result.to_string())

            Original Name                              Lookup 1  Lookup 1 Confidence  Lookup 1 Index                                             Lookup 2  Lookup 2 Confidence  Lookup 2 Index                                    Lookup 3  Lookup 3 Confidence  Lookup 3 Index                             Lookup 4  Lookup 4 Confidence  Lookup 4 Index                                                                       Lookup 5  Lookup 5 Confidence  Lookup 5 Index
0  I must be leaving now.  3498566:I must be leaving you now.\n                 0.64           22527  21481679:I'm leaving now. You're leaving with me.\n                 0.63           71945  2071944:I must be leaving now, my dears.\n                 0.62           43567  6437504:- Now. We're leaving now.\n                 0.59           71309  4268527:Mr. Hastings will be leaving now. Mr. Hastings will be leaving now.\n                 0.58           84496


In [10]:
# from itertools import islice

# def read_line_n(filename, n):
#     with open(filename, 'r') as file:
#         line = next(islice(file, n-1, n), None)
#     return line

import subprocess

def read_line_n(filename, n):
    command = f"sed -n {n}p {filename}"
    try:
        result = subprocess.check_output(command, shell=True, text=True)
        return result.strip()
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        return None
    
read_line_n('a', 10)

Error: Command 'sed -n 10p a' returned non-zero exit status 2.


sed: can't read a: No such file or directory


In [7]:
import time
data_dict = {}
k_matches = 5
with open(temp_file, 'r') as f:
    lines = f.readlines()
    start = time.time()
    unique_lines = set(lines)
    for line in unique_lines:
        number, text = line.strip().split(':', 1)
        data_dict[text] = int(number)
    end = time.time()
    print(end - start)
    result = tm.matcher(original=[example_text], lookup=data_dict.keys(), k_matches=k_matches, ngram_length=4)
    print(result.to_string())

    res = []
    for i in range(k_matches):
        text = result[f'Lookup {i + 1}'].item()
        # print(text)
        # print(data_dict.get(text))
        # print(read_line_n(file_large_2, data_dict.get(text)))
        res.append({
            'line': data_dict.get(text),
            'source_text': text,
            'target_text': read_line_n(file_large_2, data_dict.get(text)),
            'probability': result[f'Lookup {i + 1} Confidence'].item()
        })

    


0.16878342628479004
            Original Name        Lookup 1  Lookup 1 Confidence  Lookup 1 Index      Lookup 2  Lookup 2 Confidence  Lookup 2 Index            Lookup 3  Lookup 3 Confidence  Lookup 3 Index              Lookup 4  Lookup 4 Confidence  Lookup 4 Index           Lookup 5  Lookup 5 Confidence  Lookup 5 Index
0  I must be leaving now.  - Leaving now.                 0.88           44903  Leaving now.                 0.84            1549  I must be leaving.                 0.79           18463  I'll be leaving now.                 0.79           17982  're leaving now .                 0.79           59193


In [8]:
print(res)

[{'line': 33538885, 'source_text': '- Leaving now.', 'target_text': 'Idziemy.', 'probability': 0.88}, {'line': 35292179, 'source_text': 'Leaving now.', 'target_text': 'Już wyruszam.', 'probability': 0.84}, {'line': 770694, 'source_text': 'I must be leaving.', 'target_text': 'Muszę już iść.', 'probability': 0.79}, {'line': 11342999, 'source_text': "I'll be leaving now.", 'target_text': 'Pójdę już.', 'probability': 0.79}, {'line': 33384470, 'source_text': "'re leaving now .", 'target_text': 'odejć teraz.', 'probability': 0.79}]


In [15]:
import os

temp_file = 'data/temp.txt'
os.remove(temp_file)